In [61]:
import numpy as np
import scipy.stats as scp 

In [62]:
def call_option(S_, K_, r_,sigma_ ,tau_ ,d1_,d2_):
    return scp.norm.cdf(d1_) * S_ - scp.norm.cdf(d2_) * K_ * np.exp(-r_ *tau_)

def call_vega(S_, sigma_ ,tau_ , d1_):
    return S_ * scp.norm.pdf(d1_) * sqrt(tau_)

def d(S_, K_, r_,sigma_ ,tau_):
    d1_ = 1 / (sigma_ * np.sqrt(tau_)) * ( np.log(S_/K_) + (r_ + sigma_**2/2) * tau_)
    d2_ = d1_ - sigma_ * np.sqrt(tau_)
    return d1_, d2_

In [67]:
def Newton_Raphson(S_,K_,r_,tau_, sigma0_ ,price_, epsilon_):
    d1_,d2_ = d(S_, K_, r_,sigma0_ ,tau_)
    g = call_option(S_, K_, r_,sigma0_ ,tau_,d1_,d2_)-price_
    sigma_ = sigma0_ - g/call_vega(S_, sigma0_ ,tau_, d1_)
    while np.abs(sigma_-sigma0_)/sigma0_ > epsilon_:
        sigma0_ = sigma_
        d1_,d2_ = d(S_, K_, r_,sigma0_ ,tau_)
        g = call_option(S_, K_, r_,sigma0_ ,tau_,d1_,d2_)-price_
        sigma_ = sigma0_ - g/call_vega(S_, sigma0_ ,tau_, d1_)
    return sigma_


In [68]:
Newton_Raphson(194.11,210,0.01,60/365,0.5,1.50,1e-6)

0.20015597622419934

In [88]:
def Brent(S_,K_,r_,tau_, sigma0_,sigma1_,sigma2_ ,price_, epsilon_):
    volatility = [sigma0_,sigma1_,sigma2_]
    g = []
    d1_,d2_ = d(S_, K_, r_,sigma0_ ,tau_)
    g.append(call_option(S_, K_, r_,sigma0_ ,tau_,d1_,d2_)-price_)
    d1_,d2_ = d(S_, K_, r_,sigma1_ ,tau_)
    g.append(call_option(S_, K_, r_,sigma1_ ,tau_,d1_,d2_)-price_)
    d1_,d2_ = d(S_, K_, r_,sigma2_ ,tau_)
    g.append(call_option(S_, K_, r_,sigma2_ ,tau_,d1_,d2_)-price_)
    i=2
    while np.abs(volatility[i]-volatility[i-1])/volatility[i-1] > epsilon_:
        if ((volatility[i]==volatility[i-1])):
            sigma_ = volatility[i-1]-g[i-1]*(volatility[i-1]-volatility[i-2])/(g[i-1]-g[i-2])
        else :
            aux0_ = volatility[i]*g[i-1]*g[i-2]/((g[i]-g[i-1])*(g[i]-g[i-2]))
            aux1_ = volatility[i-1]*g[i-2]*g[i]/((g[i-1]-g[i-2])*(g[i-1]-g[i]))
            aux2_ = volatility[i-2]*g[i-1]*g[i]/((g[i-2]-g[i-1])*(g[i-2]-g[i]))
            sigma_ = aux0_ + aux1_ + aux2_
        volatility.append(sigma_)
        i=i+1
        d1_,d2_ = d(S_, K_, r_,volatility[i] ,tau_)
        g.append(call_option(S_, K_, r_,volatility[i] ,tau_,d1_,d2_)-price_)
    return volatility
    

In [89]:
Brent(194.11,210,0.01,60/365,0.5,0.4,0.3,1.50,1e-6)

[0.5,
 0.4,
 0.3,
 0.20937701493381328,
 0.2009149104367688,
 0.2001627676457661,
 0.20015597702227464,
 0.20015597622419853]